In [1]:
import pandas as analytics
import os
import numpy as maths

# Question 1

In [2]:
first_attribute = [3*i for i in range(1000)]
second_attribute = [2*i for i in range(1000)]
third_attribute = [5*i for i in range(1000)]

df_data = analytics.DataFrame(data = {'x1':first_attribute,'x2':second_attribute,'x3':third_attribute})
df_data = df_data.sample(frac=1).reset_index(drop=True)
df_data['y'] = (df_data['x1'] + df_data['x2'] + df_data['x3'])%3

if not os.path.exists('data') : os.mkdir('data')

df_data.to_csv('data/multinomial_distribution.csv',index=False)
df_data

,x1,x2,x3,y
0,879,586,1465,2
1,2943,1962,4905,0
2,2814,1876,4690,2
3,72,48,120,0
4,1464,976,2440,2
...,...,...,...,...
995,1047,698,1745,1
996,2208,1472,3680,1
997,984,656,1640,1
998,2292,1528,3820,2


# Question 2

In [3]:
d1 = ["send us your internet banking password","spam"]
d2 = ["send us your mobile number","ham"]
d3 = ["change your internet banking password","ham"]
d4 = ["change your userid","spam"]
d5 = ["send your internet banking password","spam"]
d6 = ["send us your account details","spam"]

documents = [d1,d2,d3,d4,d5,d6]

In [12]:

def naive_bayes(documents):
    final = ''
    for document in documents:
        document[0] = document[0] + " "
        final = final + document[0]
        
    dictionary = list(set(final.split()))
    
    outputs = []
    inputs = []
    for document in documents:
        input = []
        for word in dictionary:
            input.append(document[0].count(word + " "))
    
        inputs.append(input)
            
        if document[1] == 'spam' : outputs.append(1)
        else : outputs.append(0)        
    
    df_data = analytics.DataFrame(data = inputs, columns = dictionary)
    df_data['outputs'] = outputs
    
    df_fail = df_data[df_data['outputs'] == 0]
    df_success = df_data[df_data['outputs'] == 1]

    fail_space = len(df_fail)/len(df_data)
    success_space = len(df_success)/len(df_data)

    
    prob_fail = len(df_fail)/len(df_data)
    prob_success = 1 - prob_fail
    
    fail_word = {}
    success_word = {}
    
    for word in dictionary:
        fail_word.update({word:sum(df_fail[word])}) 
        success_word.update({word:sum(df_success[word])})
    
    fail_word_prob = {}
    success_word_prob = {}
    for key in fail_word.keys():
        fail_word_prob.update({key:(fail_word[key]/fail_space)})
        success_word_prob.update({key:(success_word[key]/success_space)})
    
    return df_data, prob_success, prob_fail, success_word, fail_word, dictionary

In [23]:
df_data, prob_success, prob_fail, success_word , fail_word , dictionary = naive_bayes(documents)

In [24]:
df_data

,change,your,send,userid,mobile,details,account,password,banking,internet,us,number,outputs
0,0,1,1,0,0,0,0,1,1,1,1,0,1
1,0,1,1,0,1,0,0,0,0,0,1,1,0
2,1,1,0,0,0,0,0,1,1,1,0,0,0
3,1,1,0,1,0,0,0,0,0,0,0,0,1
4,0,1,1,0,0,0,0,1,1,1,0,0,1
5,0,1,1,0,0,1,1,0,0,0,1,0,1


In [28]:
test_inputs = [
    "change your account number",
    "send your address"
]

test_inputs

['change your account number', 'send your address']

In [29]:
def test_naive_bayes(dictionary , test_inputs,prob_success, prob_fail, success_word, fail_word, laplace_smoothing = True):
    laplace_smoothing = int(laplace_smoothing)
    test_outputs = []
    for test_input in test_inputs:
        fail = prob_fail
        success = prob_success
        
        fail_numerators = []
        success_numerators = []
        
        for word in dictionary:
            if word in test_input:
                success_numerators.append(success_word[word] + laplace_smoothing)
                fail_numerators.append(fail_word[word] + laplace_smoothing)
                
        for i in range(len(success_numerators)):
            success = success * success_numerators[i]
            fail = fail * fail_numerators[i]
        
        success = success / sum(success_numerators)**len(success_numerators)
        fail = fail / sum(fail_numerators)**len(fail_numerators)
            
        if success > fail : test_outputs.append('spam')
        else : test_outputs.append('ham')

    df_test = analytics.DataFrame(data = test_inputs, columns = ['documents'])
    df_test['output'] = test_outputs
    return df_test

In [30]:
test_naive_bayes(dictionary , test_inputs, prob_success, prob_fail, success_word, fail_word)

,documents,output
0,change your account number,spam
1,send your address,spam


# Question 3

In [36]:
data = {
    "A1" : [1,1,0,0,0,0],
    "A2" : [0,1,1,1,0,0],
    "A3" : [0,1,0,1,1,1],
    "A4" : [1,0,1,1,0,1],
    "outputs" : [1,1,1,0,0,0]
       }

In [37]:
df_data = analytics.DataFrame(data = data )
df_data

,A1,A2,A3,A4,outputs
0,1,0,0,1,1
1,1,1,1,0,1
2,0,1,0,1,1
3,0,1,1,1,0
4,0,0,1,0,0
5,0,0,1,1,0


In [59]:
dictionary = list(df_data.columns[:-1])
df_fail = df_data[df_data['outputs'] == 0]
df_success = df_data[df_data['outputs'] == 1]

fail_space = len(df_fail)/len(df_data)
success_space = len(df_success)/len(df_data)


prob_fail = len(df_fail)/len(df_data)
prob_success = 1 - prob_fail

fail_word = {}
success_word = {}

for word in dictionary:
    fail_word.update({word:sum(df_fail[word])}) 
    success_word.update({word:sum(df_success[word])})

fail_word_prob = {}
success_word_prob = {}
for key in fail_word.keys():
    fail_word_prob.update({key:(fail_word[key]/fail_space)})
    success_word_prob.update({key:(success_word[key]/success_space)})


In [48]:
test_inputs = [[1,1,0,1],[0,0,0,0],[1,1,1,1],[1,0,1,0]]

In [68]:
laplace_smoothing = False

In [72]:
laplace_smoothing = int(laplace_smoothing)
test_outputs = []
for test_input in test_inputs:
    fail = 1
    success = 1
    
    fail_numerators = []
    success_numerators = []
    
    for word in dictionary:
        if word in test_input:
            success_numerators.append(success_word[word] + laplace_smoothing)
            fail_numerators.append(fail_word[word] + laplace_smoothing)
            
    for i in range(len(success_numerators)):
        success = success * success_numerators[i]
        fail = fail * fail_numerators[i]
    
    success = success / sum(success_numerators)**len(success_numerators)
    fail = fail / sum(fail_numerators)**len(fail_numerators)

    success = prob_success * success 
    fail = prob_fail * fail

    print(success)
    print(fail)
    
    if success > fail : test_outputs.append('spam')
    else : test_outputs.append('ham')

test_outputs
# df_test = analytics.DataFrame(data = test_inputs, columns = ['documents'])
# df_test['output'] = test_outputs

0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5


['ham', 'ham', 'ham', 'ham']

In [73]:
success_numerators

[]